# astropy.io.fits

The `astropy.io.fits` package evolved from the `pyfits` package which you may have encountered in the past.  `pyfits` has been deprected, though it is being maintained so that it keeps up with `astropy.io.fits` for now.

In [2]:
from astropy.io import fits